

### Image segementaion model train 2022 Nov




## 1. Import packages

In [ ]:
!pip install torchvision --upgrade
!pip install grad-cam
!pip install timm
!pip install imagecodecs
!pip install pytorchtools
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.datasets import VisionDataset
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset

import os 
from os import path

import numpy as np
import pandas as pd
from scipy.io import loadmat

from tqdm import tqdm
from PIL import Image

# read tiff
import zipfile
from tifffile import imread
from torchvision.transforms import ToTensor
import random
import csv
import matplotlib.pyplot as plt
import cv2 as cv

In [ ]:
from unet import UNet
from keyholeDataset import Keyhole
from loss import DiceBCEWithActivationLoss 
from augmentation import get_training_augmentation, preprocess
from utils import plot_2_sidebyside, plot_3_sidebyside, save_model, save_loss_record
from train import train
from validation import validation
import segmentation_models_pytorch as smp


## 2. Initiate a model

In [ ]:
#model = torch.hub.load('milesial/Pytorch-UNet', 'unet_carvana', pretrained=True, scale=0.5)

#model = UNet(n_channels=3, n_classes=1, bilinear=1)


# # resnet50, mobilenet_v2,
# model = smp.Unet(
#     encoder_name=None,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,                      # model output channels (number of classes in your dataset)
# )


model = smp.DeepLabV3(
    encoder_name="mobilenet_v2",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)


model_name = "Unet_Plain"

torch.cuda.empty_cache()
model.cuda()

In [ ]:
model.segmentation_head = nn.Sequential(*list(model.segmentation_head.children())[:-1])

In [ ]:
!pwd

In [ ]:
# #load model
#model_path = "/content/drive/MyDrive/DL_segmentation_models/Unet_Plain_epoch_76"
model_path = "/content/drive/MyDrive/DL_segmentation_models/DeepLabV3_MobV2_Pretrain_epoch_218"
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
# for key, value in checkpoint.items():
#     print(key)

## 3. load data + specify batch_size and epochs

In [ ]:
!mkdir Keyhole

from google.colab import drive
drive.mount('/content/drive')

with zipfile.ZipFile('/content/drive/MyDrive/DL_segmentation_data/keyhole_segmentation_data.zip', 'r') as zip:
  zip.extractall(path='/content/Keyhole')


cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
num_workers = 4 if cuda else 0
print("Cuda = " + str(cuda)+" with num_workers = "+str(num_workers))


In [ ]:
# need to write config file to make this part elegent
batch_size = 3
epochs = 300

train_dataset = Keyhole('/content/Keyhole/keyhole_segmentation_data', 
                        transform=get_training_augmentation(),
                        preprocess=None,
                        mode="train", 
                        csv_name="/image_and_split_1.csv")
val_dataset = Keyhole('/content/Keyhole/keyhole_segmentation_data', 
                      transform=None, 
                      preprocess=None, 
                      mode="val", 
                      csv_name="/image_and_split_1.csv")
test_dataset = Keyhole('/content/Keyhole/keyhole_segmentation_data', 
                       transform=None, 
                       preprocess=None, 
                       mode="test", 
                       csv_name="/image_and_split_1.csv")

print(f"Train size: {len(train_dataset)}")
print(f"Valid size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

## 4. Model eval

### 4.1 IoU

In [ ]:
SMOOTH = 1e-6

# def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
#     # You can comment out this line if you are passing tensors of equal shape
#     # But if you are passing output from UNet or something it will most probably
#     # be with the BATCH x 1 x H x W shape
#     outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
#     intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
#     union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
    
#     iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
#     thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
#     return thresholded  # Or thresholded.mean() if you are interested in average across the batch
    
  
#   # Numpy version
# # Well, it's the same function, so I'm going to omit the comments

def iou_numpy(outputs: np.array, labels: np.array):
    outputs = outputs #.squeeze(1)
    
    intersection = (outputs & labels).sum((0, 1))
    union = (outputs | labels).sum((0, 1))
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)
    
    #thresholded = np.ceil(np.clip(20 * (iou - 0.5), 0, 10)) / 10
    
    return iou  # Or thresholded.mean()
    

In [ ]:
images = []
true_masks = []
pred_masks = []
iou_record = []

for i, batch in enumerate(test_loader):
      print("i = ", i)
      x = batch['image'].float().to(device)  #.float().to(device) 
      y = batch['mask'].float().to(device) 
      assert(len(x) == len(y))
      yp = model(x)
      #print("x shape", x.shape) #torch.Size([1, 3, 572, 572])
      #print("y shape", y.shape) #torch.Size([1, 1, 572, 572])


      for i in range(len(x)):
        x_ = x[i].unsqueeze(0)
        y_ = y[i].unsqueeze(0)
        yp_ = yp[i]

        image = x_.detach().cpu().numpy()[0][0].astype(int) #(576,576)
        images.append(image)
        true_mask = y_.detach().cpu().numpy()[0][0].astype(int)
        true_masks.append( true_mask)
        pred_mask = (yp_.detach().cpu().numpy()[0]>0).astype(int)
        pred_masks.append(pred_mask)
        
        assert(true_mask.shape == pred_mask.shape)
        iou_score = iou_numpy(pred_mask, true_mask)
        #plot_2_sidebyside(true_mask, pred_mask)
        print("iou: ", iou_score)
        iou_record.append(iou_score)

In [ ]:
np.mean(iou_record)

### 4.2 IoU of different materials on test data

In [ ]:
# plt.figure(figsize=(10,10))
# plt.plot(train_loss_record)
# plt.plot(val_loss_record)

In [ ]:
csv_path = "/content/Keyhole/keyhole_segmentation_data/image_and_split_1.csv"
df = pd.read_csv(csv_path)

In [ ]:
test_df = df[df['train_val_test_index']==2]

In [ ]:
test_df['iou'] = iou_record

In [ ]:
test_df['image']

In [ ]:
def label_material (row):
  name = row['image']
  if 'AA6061' in name:
    return 'AA6061'
  if 'SS316' in name:
    return 'SS316'
  if 'IN718' in name:
    return 'IN718'
  if 'Ti64S' in name:
    return 'Ti64S'
  if 'Ti64' in name:
    return 'Ti64'
  else:
    return 'Other'

In [ ]:
test_df['material'] = test_df.apply (lambda row: label_material(row), axis=1)


In [ ]:
test_df.head()

In [ ]:
test_df.groupby('material').mean()

In [ ]:
test_df.groupby('material').std()

### 4.3 FP and FN superimposed on an image

In [ ]:
from skimage.color import gray2rgb

In [ ]:
def visualize_prediction_accuracy(prediction, truth):
    out = np.zeros(truth.shape, dtype='uint8') #(576, 576, 3)
    print(out.shape)
    t = np.all(truth == 1, axis=-1)
    print
    # preds = #[prediction[:,:,i] for i in range(prediction.shape[2])]
    p = prediction

    out[t & p, :] = [255, 255, 255] # true positive
    out[t & ~p, :] = [255, 0, 255] # false negative
    out[~t & p, :] = [0, 255, 255] # false positive       
    return out


In [ ]:
i=0
true_mask = gray2rgb(true_masks[i]) #(576, 576, 3)

pred_mask = pred_masks[i] # (576, 576)

out = np.zeros(true_mask.shape, dtype='uint8') #(576, 576, 3)
t = np.all(true_mask == 1, axis=-1) #(576, 576)
p = pred_mask # (576, 576)

channels = [t & p, t & ~p, ~t & p] # true positive (white), false negative (pink), false positive (blue)
colors = [[255, 255, 255], [255, 0, 255], [0, 255, 255]]

# ch = channels[0]
# color = colors[0]
for n in range(3):

  ch = channels[n]
  color = colors[n]

  for i in range(ch.shape[0]):
    for j in range(ch.shape[1]):
      if ch[i, j] == 1:
        out[i,j,:] = color


In [ ]:
plt.imshow(out[100:300, 200:450, :])